# Setup & Environment Verification — DreamScape AI

This notebook confirms that the environment is correctly configured, the **DreamBank Annotated** dataset loads from Hugging Face, and basic EDA works.

**It will:**
- Print library versions and runtime details
- Load a small sample of the dataset
- Run light cleaning and generate one plot
- Save a small processed JSON artifact for the project

Outputs saved to:
- `results/eda_plots.png`
- `data/processed/dreambank_clean.json`

In [1]:
# Core imports and version check
import sys, os, json, shutil, warnings, pathlib, platform
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib, seaborn
from datasets import load_dataset

# Optional torch presence
try:
    import torch
    TORCH_OK = True
except Exception:
    TORCH_OK = False

print("✅ Environment check")
print("- OS:", platform.platform())
print("- Python:", sys.version.split()[0])
print("- pandas:", pd.__version__)
print("- matplotlib:", matplotlib.__version__)
print("- seaborn:", seaborn.__version__)
print("- datasets:", load_dataset.__module__.split('.')[0], "(🤗 datasets)")

if TORCH_OK:
    print("- torch:", torch.__version__)
    print("  CUDA available:", torch.cuda.is_available())
else:
    print("- torch: not installed (OK for EDA)")

✅ Environment check
- OS: macOS-15.6.1-arm64-arm-64bit
- Python: 3.11.13
- pandas: 2.3.3
- matplotlib: 3.10.7
- seaborn: 0.13.2
- datasets: datasets (🤗 datasets)
- torch: 2.8.0
  CUDA available: False


In [2]:
# Ensure repo structure paths exist (idempotent)
BASE = Path(".").resolve()
DATA_RAW = BASE / "data" / "raw"
DATA_PROC = BASE / "data" / "processed"
RESULTS = BASE / "results"

for p in [DATA_RAW, DATA_PROC, RESULTS]:
    p.mkdir(parents=True, exist_ok=True)

# quick write permission check
testfile = BASE / "_perm_test"
with open(testfile, "w") as f:
    f.write("ok")
testfile.unlink()
print("✅ Write permissions OK in", BASE)

✅ Write permissions OK in /Users/saturnine/Dreamscape
